# Baseline Model

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, LassoCV, LogisticRegression, LogisticRegressionCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
from sklearn.preprocessing import OneHotEncoder 
from sklearn.metrics import roc_curve, roc_auc_score, r2_score, mean_squared_error

np.random.seed(42)

## Baseline Model: Linear Regression, with Feature Engineering
For our baseline model, we performed feature engineering to compress time series data into aggregate information (e.g. average acceleration, std of acceleration), in order to transform into tabular data. 

Some of the assumptions are violated: 
- data are not iid. Same patients have multiple observations.

In [ ]:
X_train_orig = pd.read_csv("processed_data/X_train_original.csv", index_col="Unnamed: 0")
X_test_orig = pd.read_csv("processed_data/X_test_original.csv",  index_col="Unnamed: 0")

In [ ]:
X_train = pd.read_csv("processed_data/X_train_final.csv", index_col = "Unnamed: 0")
X_test = pd.read_csv("processed_data/X_test_final.csv", index_col = "Unnamed: 0")

In [ ]:
y_train = pd.read_csv("processed_data/y_train.csv",index_col = "Unnamed: 0")['on_off']
y_test = pd.read_csv("processed_data/y_test.csv",index_col = "Unnamed: 0")['on_off']

In [ ]:
# check correlation matrix of features
# help from: https://towardsdatascience.com/better-heatmaps-and-correlation-matrix-plots-in-python-41445d0f2bec

corr = X_train_orig.drop(columns=["subject_id","measurement_id"]).corr()
plt.figure(figsize=(10,10))
plt.title("correlation matrix heatmap", fontsize=18)
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap='bwr',
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

There are highly correlated variables, so regularization is required.

In [ ]:
# drop irrelevant features
X_train.drop(columns=["subject_id","measurement_id"],inplace=True)
X_test.drop(columns=["subject_id","measurement_id"],inplace=True)
X_train.shape, X_test.shape

In [ ]:
# dataset without patient id information (cluster)
X_train_nopatient = X_train.drop(columns=['cluster1','cluster2', 'cluster3'])
X_test_nopatient = X_test.drop(columns=['cluster1','cluster2', 'cluster3'])

In [ ]:
X_train.shape, y_train.shape

## 1.A Generalized Linear Regression (= no patient cluster information)

In [ ]:
# try linear regression, without patient id (cluster) information
linreg = LinearRegression().fit(X_train_nopatient, y_train)

In [ ]:
def get_scores(model, X_train, y_train, X_test, y_test):
    rmse_train = np.sqrt(mean_squared_error(y_train, model.predict(X_train)))
    rmse_test = np.sqrt(mean_squared_error(y_test, model.predict(X_test)))
    rmse_cv = np.sqrt(-np.mean(cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_error', cv=5)))
    print("Train RMSE:", rmse_train)
    print("CV RMSE:", rmse_cv)
    print("Test RMSE:", rmse_test)
    return rmse_train, rmse_cv, rmse_test

In [ ]:
print("Generalized linear regression")
rmse_train_linreg, rmse_cv_linreg, rmse_test_linreg = get_scores(linreg, X_train_nopatient, y_train, X_test_nopatient, y_test)

RMSE shares same unit as our ordinal response. This in general means that we are off by 1. 

In [ ]:
print(linreg.predict(X_test_nopatient))
print(y_test.values)

## 1.B Personalized Linear Regression (= with patient cluster information)

In [ ]:
# try linear regression, without patient id (cluster) information
linregper = LinearRegression().fit(X_train, y_train)

In [ ]:
print("Personalized linear regression")
rmse_train_linregper,rmse_cv_linregper, rmse_test_linregper = get_scores(linregper, X_train, y_train, X_test, y_test)

## 2.A Generalized Linear Regression with Lasso regularization

## 2.B Personalized Linear Regression with Lasso regularization

## 3.A Generalized Random Forest Regressor

We tried Random forest regressor, which is a nonlinear model. 

In [ ]:
# help from: https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

param_grid = {
    'max_depth': [2,5,10,20,50,100],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [10, 50, 100, 200]
}

In [ ]:
# Create a based model
rf = RandomForestRegressor(max_features='sqrt')
# Instantiate the grid search model
generalized_rf = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)
generalized_rf.fit(X_train_nopatient, y_train)

In [ ]:
generalized_rf.best_params_

In [ ]:
print("Generalized Random Forest Regressor")
rmse_train_rf, rmse_cv_rf, rmse_test_rf = get_scores(generalized_rf, X_train_nopatient, y_train, X_test_nopatient, y_test)

## 3.B Personalized Random Forest Regressor

In [ ]:
# Create a based model
rf = RandomForestRegressor(max_features='sqrt')
# Instantiate the grid search model
personalized_rf = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)
personalized_rf.fit(X_train, y_train)

In [ ]:
personalized_rf.best_params_

In [ ]:
print("Personalized Random Forest Regressor")
rmse_train_rfper, rmse_cv_rfper, rmse_test_rfper = get_scores(personalized_rf, X_train, y_train, X_test, y_test)

### Error Analysis: gender stratification

In [ ]:
X_test['on_off'] = y_test
data_test_male = X_test.loc[X_test['is_male'] == 1]
y_test_male = data_test_male['on_off']
X_test_male = data_test_male.drop(columns=['on_off'])

In [ ]:
print("Test RMSE for male patients:")
np.sqrt(mean_squared_error(y_test_male, personalized_rf.predict(X_test_male)))

In [ ]:
X_test['on_off'] = y_test
data_test_female = X_test.loc[X_test['is_male'] == 0]
y_test_female = data_test_female['on_off']
X_test_female = data_test_female.drop(columns=['on_off'])

In [ ]:
print("Test RMSE for female patients:")
np.sqrt(mean_squared_error(y_test_female, personalized_rf.predict(X_test_female)))

### Error Analysis: label stratification

In [ ]:
X_test['on_off'] = y_test
data_test = X_test.loc[X_test['on_off'] == 0]
y_test0 = data_test['on_off']
X_test0 = data_test.drop(columns=['on_off'])

In [ ]:
print("Test RMSE for medication status=0:")
np.sqrt(mean_squared_error(y_test0, personalized_rf.predict(X_test0)))

In [ ]:
X_test['on_off'] = y_test
data_test = X_test.loc[X_test['on_off'] == 1]
y_test1 = data_test['on_off']
X_test1 = data_test.drop(columns=['on_off'])

In [ ]:
print("Test RMSE for medication status=1:")
np.sqrt(mean_squared_error(y_test1, personalized_rf.predict(X_test1)))

In [ ]:
X_test['on_off'] = y_test
data_test = X_test.loc[X_test['on_off'] == 2]
y_test2 = data_test['on_off']
X_test2 = data_test.drop(columns=['on_off'])

In [ ]:
print("Test RMSE for medication status=2:")
np.sqrt(mean_squared_error(y_test2, personalized_rf.predict(X_test2)))

In [ ]:
X_test['on_off'] = y_test
data_test = X_test.loc[X_test['on_off'] == 3]
y_test3 = data_test['on_off']
X_test3 = data_test.drop(columns=['on_off'])

In [ ]:
print("Test RMSE for medication status=3:")
np.sqrt(mean_squared_error(y_test3, personalized_rf.predict(X_test3)))

In [ ]:
X_test['on_off'] = y_test
data_test = X_test.loc[X_test['on_off'] == 4]
y_test4 = data_test['on_off']
X_test4 = data_test.drop(columns=['on_off'])

In [ ]:
print("Test RMSE for medication status=4:")
np.sqrt(mean_squared_error(y_test4, personalized_rf.predict(X_test4)))

### Error Analysis: individuals

In [ ]:
# try lasso regression, without patient id (cluster) information

lasso = LassoCV(cv=5).fit(X_train_nopatient, y_train)

In [ ]:
np.sqrt(mean_squared_error(y_train, lasso.predict(X_train_nopatient)))

In [ ]:
lasso.predict(X_train_nopatient)

In [ ]:
mean_squared_error(y_test, lasso.predict(X_test_nopatient))

In [ ]:
lasso.coef_

In [ ]:
lasso.predict(X_test_nopatient)

In [ ]:
# try lasso regression, with patient id (cluster) information

lasso_per = LassoCV(cv=5).fit(X_train, y_train)

In [ ]:
lasso_logreg = LogisticRegressionCV(cv=5, multi_class='ovr',penalty='l1',solver='liblinear',max_iter=1000).fit(X_train_nopatient,y_train)
lasso_acc = lasso_logreg.score(X_test_nopatient, y_test)
print("baseline model (lasso) accuracy")
print(f"train: {lasso_logreg.score(X_train_nopatient, y_train)*100:.2f}%")
print(f"CV score (k=5): {np.mean(cross_val_score(lasso_logreg, X_train_nopatient, y_train, cv=5))*100:.2f}%")
print(f"test: {lasso_acc*100:.3f}%") 

In [ ]:
# try without patient id, cluster information
logreg = LogisticRegression(multi_class='ovr', penalty='none', solver='lbfgs',max_iter=1000).fit(X_train_nopatient,y_train)
baseline_acc = logreg.score(X_test_nopatient, y_test)
print("baseline model accuracy")
print(f"train: {logreg.score(X_train_nopatient, y_train)*100:.2f}%")
print(f"CV score (k=5): {np.mean(cross_val_score(logreg, X_train_nopatient, y_train, cv=5))*100:.2f}%")
print(f"test: {baseline_acc*100:.2f}%")

In [ ]:
logreg.classes_

In [ ]:
logreg.predict_proba(X_test_nopatient)

In [ ]:
logreg.predict(X_test_nopatient)

In [ ]:
y_train[y_train['on_off'] == 0].shape[0]/y_train.shape[0]

In [ ]:
y_test[y_test['on_off'] == 0].shape[0]/y_test.shape[0]

Baseline model (without regularization, without patient id information) has very low performance, it's almost similar to random coin toss.

In [ ]:
logreg.coef_

In [ ]:
logreg.intercept_

In [ ]:
lasso_logreg = LogisticRegressionCV(cv=5,penalty='l1',solver='liblinear',max_iter=1000).fit(X_no_patient,y_train)
lasso_acc = lasso_logreg.score(X_no_patient_test, y_test)
print("baseline model (lasso) accuracy")
print(f"train: {lasso_logreg.score(X_no_patient, y_train)*100:.2f}%")
print(f"CV score (k=5): {np.mean(cross_val_score(lasso_logreg, X_no_patient, y_train, cv=5))*100:.2f}%")
print(f"test: {lasso_acc*100:.3f}%") 

In [ ]:
lasso_logreg.C_

Lasso regularization certainly helps a bit in model performance, by reducing multicollinearity issues.

## Principal Component Analysis

We wanted to visualize how the data are clustered by reducing the dimensionality of the features through PCA. Specifically, we wanted to see if building a personalized model is a valid approach. Are data grouped by individual patients (even when patient id information is removed)? And how separable are the medication status labels?

In [ ]:
pca = PCA(n_components=2).fit(X_train_nopatient)
print(f"Total % of variance explained with 2 components: {pca.explained_variance_ratio_.sum()*100:.2f}%")
pca = pca.transform(X_train_nopatient)

Total % of variance explained is not very high. We need to keep in mind that our 2D plots are not capturing all of the variation in the data. 

In [ ]:
plt.figure(figsize=(8,8))
sns.scatterplot(pca[:,0], pca[:,1], hue=y_train.values.reshape(y_train.shape[0],), palette=["red",'orange','green','blue','black'], alpha=0.5)
plt.title("PCA first 2 components (grouped by medication status)", fontsize=17)
plt.xlabel("1st principal component", fontsize=14)
plt.ylabel("2nd principal component", fontsize=14)
plt.legend(loc="best")

plt.tight_layout()
plt.show()

From this visualization, we can see that the medication labels are not linearly separable. If anything, nonlinear models would work better.

In [ ]:
X_train_orig = X_train_orig.astype({'subject_id': 'object'})

plt.figure(figsize=(8,8))
sns.scatterplot(pca[:,0], pca[:,1], hue = X_train_orig.subject_id, 
                palette=sns.color_palette("hls", 15),
                #size=y_train, sizes=(100, 10),
                alpha=.8)
plt.title("PCA first 2 components on CIS-PD patients", fontsize=17)
plt.xlabel("1st principal component", fontsize=14)
plt.ylabel("2nd principal component", fontsize=14)
plt.legend(loc="best")

plt.tight_layout()
plt.show()

Surely seems like the data are clustered by patients, which suggests that having a personalized model may be more suitable. It is reasonable to assume that individual patients will have differing movement behavior patterns. Since we also won't know the disease progression state of the patient, clustering may help group similar patients together. (Something to keep in mind is that we are not necessarily grouping "patients" together, but by similar 20-minute interval blocks together. Each datapoint is not an individual patient, but a patient's unique 20-minute block.) 

But if we were to incorporate patient's identity for personalized model, how do we incorporate that information into the model? One-hot encoding won't take into account new patient ids in the future.
Some ways: use embedding, or use clustering to find similar patient groups, and input the cluster information instead.
(https://datascience.stackexchange.com/questions/37480/how-can-i-do-classification-with-categorical-data-which-is-not-fixed)

## K-means clustering

In [ ]:
# K-means clustering help from: https://pythonprogramminglanguage.com/kmeans-elbow-method/
# BUT Kmeans clustering is not ideal for high dimensional data

# k means determine k
distortions = []
K = range(1,16)
for k in K:
    kmeans_cluster = KMeans(n_clusters=k).fit(X_no_patient)
    distortions.append(sum(np.min(cdist(X_no_patient, kmeans_cluster.cluster_centers_, 'euclidean'), axis=1)) / X_train.shape[0])

# Evaluation 1: Elbow method
plt.figure(figsize=(8,8))
plt.plot(K, distortions, 'bx-')
plt.xlabel('k', fontsize=15)
plt.ylabel('Distortion',fontsize=15)
plt.title('The Elbow Method showing the optimal k', fontsize=18)
plt.tight_layout()
plt.show()

It seems like there is an elbow around k=5,6. We can cluster patients by these number of groups, and feed that cluster information for training personalized model.

In [ ]:
# Evaluation 2: silhouette score

from sklearn.metrics import silhouette_score

scores = [0]
for i in range(2,16):
    fitx = KMeans(n_clusters=i, init='random', n_init=10, n_jobs=4).fit(X_no_patient)
    score = silhouette_score(X_no_patient, fitx.labels_)
    scores.append(score)
    
plt.figure(figsize=(8,8))
plt.plot(range(1,16), np.array(scores), 'bx-')
plt.xlabel('$k$', fontsize=15)
plt.ylabel('Average Silhouette', fontsize=15)
plt.title('The Silhouette Method for $K$-means Clustering', fontsize=18)
plt.tight_layout()
plt.show()

Silhouette score reports best k = 2, and k = 4.

In [ ]:
# Evaluation 3: Gap statistics
from gap_statistic import OptimalK
from sklearn.datasets import make_blobs

gs_obj = OptimalK()

_ = gs_obj(X_no_patient, n_refs=400, cluster_array=np.arange(1, 16))

def plot_gap(gap_mat):
    gaps = gap_mat["gap_value"].values
    diffs = gap_mat["diff"]
    
    err_bars = np.zeros(len(gap_mat))
    err_bars[1:] = diffs[:-1] - gaps[:-1] + gaps[1:]

    plt.figure(figsize=(8,8))
    plt.scatter(gap_mat["n_clusters"], gap_mat["gap_value"])
    plt.errorbar(gap_mat["n_clusters"], gap_mat["gap_value"], yerr=err_bars, capsize=6)
    plt.xlabel("k", fontsize=15)
    plt.ylabel("Gap Statistic", fontsize=15)
    plt.title("Gap Statistics method for KMeans clustering", fontsize=18)
    plt.tight_layout()
    plt.show()
    
plot_gap(gs_obj.gap_df)


Gap statistics method reports optimal k=5.

In [ ]:
# visualize cluster results with PCA
best_ks = [2,4,5]
for best_k in best_ks:
    kmeans_cluster = KMeans(n_clusters=best_k).fit(X_no_patient)

    plt.figure(figsize=(8,8))
    sns.scatterplot(pca[:,0], pca[:,1], hue=kmeans_cluster.labels_, 
                    palette=sns.color_palette("hls", best_k), alpha=0.5, legend="full")
    plt.title(f"KMeans clustering (k={best_k}) results on PCA first 2 components, CIS-PD", fontsize=15)
    plt.xlabel("1st principal component", fontsize=14)
    plt.ylabel("2nd principal component", fontsize=14)
    plt.legend(loc="best")
    plt.tight_layout()
    plt.show()

Grouping patients into 2 or 4 clusters seems to be most reasonable, at least based on the limited 2D PCA plot.

In [ ]:
kmeans = KMeans(n_clusters=4).fit(X_no_patient)
X_train_cluster_label = kmeans.labels_

# compare differences of features by groups?
X_train_cp = X_train.copy()
X_train_cp['cluster'] = kmeans.labels_

In [ ]:
X_train_cp.groupby('cluster').describe()

In [ ]:
X_train_cp.groupby('cluster').mean()

The average variable values are different by different clusters. Looks promising!

## Baseline Model (with patient cluster information)

Now, we tried fitting a model with cluster information of patients.

In [ ]:
X_train_cluster_label = kmeans.labels_
X_test_cluster_label = kmeans.predict(X_no_patient_test)

In [ ]:
# create dummy variables for cluster class 
onehotencoder = OneHotEncoder(drop='first', categories='auto') 

# is_1, is_2, is_3
X_train_cluster_dummy = onehotencoder.fit_transform(X_train_cluster_label.reshape(-1,1)).toarray() 
X_test_cluster_dummy = onehotencoder.transform(X_test_cluster_label.reshape(-1,1)).toarray()


# append dummy variables of kmeans clustering labels
X_train_scaled_cluster = np.hstack((X_no_patient, X_train_cluster_dummy))
X_test_scaled_cluster = np.hstack((X_no_patient_test, X_test_cluster_dummy))

In [ ]:
logreg2 = LogisticRegression(penalty='none',solver='lbfgs',max_iter=1000).fit(X_train_scaled_cluster,y_train)
baseline2_acc = logreg2.score(X_test_scaled_cluster, y_test)
print("baseline model accuracy (with patient cluster information)")
print(f"train: {logreg2.score(X_train_scaled_cluster, y_train)*100:.2f}%")
print(f"CV score (k=5): {np.mean(cross_val_score(logreg2, X_train_scaled_cluster, y_train, cv=5))*100:.2f}%")
print(f"test: {baseline2_acc*100:.2f}%")

In [ ]:
lasso2 = LogisticRegressionCV(cv=5,penalty='l1',solver='liblinear',max_iter=1000).fit(X_train_scaled_cluster,y_train)
lasso2_acc = lasso2.score(X_test_scaled_cluster, y_test)
print("baseline model (lasso) accuracy")
print(f"train: {lasso2.score(X_train_scaled_cluster, y_train)*100:.2f}%")
print(f"CV score (k=5): {np.mean(cross_val_score(lasso2, X_train_scaled_cluster, y_train, cv=5))*100:.2f}%")
print(f"test: {lasso2_acc*100:.3f}%") # very low performance, it's almost similar to random coin toss.

In [ ]:
lasso2.C_

There wasn't much improvement even with clustering labels. Perhaps having 15 patients is too small a sample to derive any meaningful information from the clusters.

In [ ]:
lasso2.coef_.shape

In [ ]:
lasso2_coef_dict = {}
cols = ['age','avg_X', 'std_X', 'min_X', 'max_X', 'range_X',
       'avg_Y', 'std_Y', 'min_Y', 'max_Y', 'range_Y', 'avg_Z', 'std_Z',
       'min_Z', 'max_Z', 'range_Z', 'is_male', 'cluster1', 'cluster2', 'cluster3']
for col, coef in zip(cols, lasso2.coef_[0]):
    lasso2_coef_dict[col] = coef

print("Feature importance, by lasso coefficients:")
{k: v for k, v in sorted(lasso2_coef_dict.items(), key=lambda item: item[1], reverse=True)}
# keep in mind that lasso regularization arbitrarily chooses some highly correlated variables to drop to zero.
# variable interpretation is not too meaningful.

In [ ]:
# help from: https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/
# calculate roc curve - generalized baseline model
probs = lasso_logreg.predict_proba(X_no_patient_test)[:,1]
fpr, tpr, thresholds = roc_curve(y_test, probs)

In [ ]:
# calculate AUC
auc = roc_auc_score(y_test, probs)
print('AUC: %.3f' % auc)

In [ ]:
# calculate roc curve - personalized baseline model
probs2 = lasso2.predict_proba(X_test_scaled_cluster)[:,1]
fpr2, tpr2, thresholds = roc_curve(y_test, probs)

In [ ]:
# calculate AUC
auc = roc_auc_score(y_test, probs)
print('AUC: %.3f' % auc)

In [ ]:
# AUC plot - personalized model

ns_probs = [0 for _ in range(len(y_test))]
ns_fpr, ns_tpr, thresholds = roc_curve(y_test, ns_probs)

plt.figure(figsize=(8,8))
plt.plot(fpr2, tpr2, c="red", marker='.', label="personalized baseline model")
plt.plot(ns_fpr, ns_tpr, c="blue", linestyle='--', label="random")
plt.legend(loc="best",fontsize=15)
plt.xlabel('False Positive Rate',fontsize=15)
plt.ylabel('True Positive Rate',fontsize=15)
plt.tight_layout()
plt.show()

In [ ]:

pyplot.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
pyplot.plot(lr_fpr, lr_tpr, marker='.', label='Logistic')
# axis labels

# show the legend
pyplot.legend()
# show the plot
pyplot.show()

In [ ]:
# calculate accuracy by patients?

In [ ]:
# add diskynesia/tremor symptoms for maybe better clustering?

In [ ]:
# another method of personalized model: random effects model

In [ ]:
X_no_patient = pd.DataFrame(data=X_no_patient,
             columns=cols)
X_no_patient.head()

In [ ]:
# import statsmodels.api as sm
# import statsmodels.formula.api as smf

# data_temp = X_no_patient.copy()
# data_temp['on_off'] = y_train

# md = smf.mixedlm("on_off ~ age", data=data_temp, groups=X_train["subject_id"])
# mdf = md.fit()
# print(mdf.summary())